# NN debug

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2021/blob/master/04-debug/homework.ipynb)

В этой тетрадке мы рассмотрим несколько проблем с обучением сеток и способы их решения.

*Лучше решать эту домашку в колабе*

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torch.utils.tensorboard import SummaryWriter

# Data

Для обучения сеток мы будем использовать MNIST.

Качаем архив [Google Drive](https://drive.google.com/file/d/1xo-AIG2E6cTZbWGti1A5lp5FDtf4aHx_/view?usp=sharing). 
Его структура следующая:
- /
    - /train.csv
    - /val.csv
    - /train/{image_name}.png
    - /val/{image_name}.png

CSV файлы содержат название файла и его лейбл: image_name, label.

Распакуйте архив в текущую папку:
`unzip -q ./mnist_data2.zip -d ./`

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, images_dir_path: str,
                 description_csv_path: str):
        super().__init__()
        
        self.images_dir_path = images_dir_path
        self.description_df = pd.read_csv(description_csv_path,
                                           dtype={'image_name': str, 'label': int})

    def __len__(self):
        return len(self.description_df)
    
    def __getitem__(self, index):
        img_name, label = self.description_df.iloc[index, :]
        
        img_path = Path(self.images_dir_path, f'{img_name}.png')
        img = self._read_img(img_path)
        
        return dict(sample=img, label=label)
    
    @staticmethod
    def _read_img(img_path: Path):
        img = cv2.imread(str(img_path.resolve()))
        img = img.astype(np.float32)
        img = np.transpose(img, (2, 0, 1))
        
        return img

## Задание 1
**(0.4 балла)** Запустите обучение сети в ячейках ниже. За 10 эпох метрика на валидации вырастает всего до ~0.15.

*Вопросы:*
1. Почему сетка так плохо учится?
1. Найдите ошибку в коде и объясните ошибка вызывает подобное поведение в обучении?

*Requirements:*
1. Напишите ответы в markdown ячейке перед следующим заданием
1. В следующей ячейке (после вашего ответа) вставьте код с исправлением ошибки.

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = resnet18()
        self.net.fc = nn.Linear(512, 10)

    def forward(self, x):
        return self.net(x)

    def compute_all(self, batch):
        x = batch['sample'] / 255.0
        y = batch['label']
        logits = self.net(x)

        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(axis=1) == y).float().mean().cpu().numpy()
        metrics = dict(acc=acc)

        return loss, metrics


class Trainer:
    def __init__(self, model: nn.Module,
                 optimizer,
                 train_dataset: Dataset,
                 val_dataset: Dataset,
                 tboard_log_dir: str,
                 batch_size: int = 128):
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0
        self.log_writer = SummaryWriter(log_dir=tboard_log_dir)

    def train(self, num_epochs: int):
        model = self.model
        optimizer = self.optimizer

        train_loader = DataLoader(self.train_dataset, shuffle=False, batch_size=self.batch_size)
        val_loader = DataLoader(self.val_dataset, shuffle=False, batch_size=self.batch_size)
        best_loss = float('inf')

        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                for k, v in details.items():
                    self.log_writer.add_scalar(k, v, global_step=self.global_step)
                self.global_step += 1

            model.eval()
            val_losses, val_metrics_list = [], []
            for batch in tqdm(val_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                val_losses.append(loss.item())
                val_metrics_list.append(details['acc'].item())

            val_loss, val_metrics = np.mean(val_losses), np.mean(val_metrics_list)
            self.log_writer.add_scalar('val/loss', val_loss, global_step=self.global_step)
            self.log_writer.add_scalar('val/metrics', val_metrics, global_step=self.global_step)

In [ ]:
mnist_train = MNISTDataset(images_dir_path='./mnist_data/train/',
                           description_csv_path='./mnist_data/train.csv')
mnist_val = MNISTDataset(images_dir_path='./mnist_data/val/',
                         description_csv_path='./mnist_data/val.csv')

model = ResNet18()
opt = optim.SGD(model.parameters(), lr=1e-2)

trainer = Trainer(model=model, optimizer=opt, train_dataset=mnist_train,
                  val_dataset=mnist_val, tboard_log_dir='./tboard_logs/exp1')

In [ ]:
trainer.train(10)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./tboard_logs

## Задание 2
**(0.2 балла)** Запустите обучение сети в ячейках ниже. За 10 эпох сетка не покажет качества выше случайного угадывания.

*Вопросы:*
1. Почему сетка так плохо учится?
1. Найдите ошибку в коде и объясните почему найденная ошибка вызывает подобное поведение в обучении?

*Requirements:*
1. Напишите ответы в markdown ячейке перед следующим заданием
1. В следующей ячейке (после вашего ответа) вставьте код с исправлением ошибки.

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = resnet18()
        self.net.fc = nn.Linear(512, 10)

    def forward(self, x):
        return self.net(x)

    def compute_all(self, batch):
        x = batch['sample'] / 255.0
        y = batch['label']
        logits = self.net(x)

        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(axis=1) == y).float().mean().cpu().numpy()
        metrics = dict(acc=acc)

        return loss, metrics


class Trainer:
    def __init__(self, model: nn.Module,
                 optimizer,
                 train_dataset: Dataset,
                 val_dataset: Dataset,
                 tboard_log_dir: str,
                 batch_size: int = 128):
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0
        self.log_writer = SummaryWriter(log_dir=tboard_log_dir)

    def train(self, num_epochs: int):
        model = self.model
        optimizer = self.optimizer

        train_loader = DataLoader(self.train_dataset, shuffle=True, batch_size=self.batch_size)
        val_loader = DataLoader(self.val_dataset, shuffle=False, batch_size=self.batch_size)
        best_loss = float('inf')

        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                for k, v in details.items():
                    self.log_writer.add_scalar(k, v, global_step=self.global_step)
                self.global_step += 1

            model.eval()
            val_losses, val_metrics_list = [], []
            for batch in tqdm(val_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                val_losses.append(loss.item())
                val_metrics_list.append(details['acc'].item())

            val_loss, val_metrics = np.mean(val_losses), np.mean(val_metrics_list)
            self.log_writer.add_scalar('val/loss', val_loss, global_step=self.global_step)
            self.log_writer.add_scalar('val/metrics', val_metrics, global_step=self.global_step)

In [ ]:
mnist_train = MNISTDataset(images_dir_path='./mnist_data/train/',
                           description_csv_path='./mnist_data/train.csv')
mnist_val = MNISTDataset(images_dir_path='./mnist_data/val/',
                         description_csv_path='./mnist_data/val.csv')

model = ResNet18()
opt = optim.SGD(model.parameters(), lr=10e-2, weight_decay=9e-1)

trainer = Trainer(model=model, optimizer=opt, train_dataset=mnist_train,
                  val_dataset=mnist_val, tboard_log_dir='./tboard_logs/exp2')

In [ ]:
trainer.train(10)

In [ ]:
%tensorboard --logdir ./tboard_logs

## Задание 3
**(0.4 балла)** Запустите обучение сети в ячейках ниже. В сети будут использоваться предобученные параметры, которые должны были помочь выдавать качество около 1. Однако, за 5 эпох сетка не выдаст качество, которое мы ожидали.

Перед запуском ячеек скачайте используемое состояние модели [pretrained_model.pt](https://drive.google.com/file/d/1JITAz1L8mWpTGany84YMYKIhzVgsBf_9/view?usp=sharing).

*Вопросы:*
1. Почему сетка так плохо учится?
1. Найдите ошибку и объясните почему найденная ошибка вызывает подобное поведение в обучении?

*Requirements:*
1. Напишите ответы в markdown ячейке после ячейки с тензорбордом.
1. В следующей ячейке (после вашего ответа) вставьте код с исправлением ошибки.

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = resnet18()
        self.net.fc = nn.Linear(512, 10)

    def forward(self, x):
        return self.net(x)

    def compute_all(self, batch):
        x = batch['sample'] / 255.0
        y = batch['label']
        logits = self.net(x)

        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(axis=1) == y).float().mean().cpu().numpy()
        metrics = dict(acc=acc)

        return loss, metrics


class Trainer:
    def __init__(self, model: nn.Module,
                 optimizer,
                 train_dataset: Dataset,
                 val_dataset: Dataset,
                 tboard_log_dir: str,
                 batch_size: int = 128):
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0
        self.log_writer = SummaryWriter(log_dir=tboard_log_dir)

    def train(self, num_epochs: int):
        model = self.model
        optimizer = self.optimizer

        train_loader = DataLoader(self.train_dataset, shuffle=True, batch_size=self.batch_size)
        val_loader = DataLoader(self.val_dataset, shuffle=False, batch_size=self.batch_size)
        best_loss = float('inf')

        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                for k, v in details.items():
                    self.log_writer.add_scalar(k, v, global_step=self.global_step)
                self.global_step += 1

            model.eval()
            val_losses, val_metrics_list = [], []
            for batch in tqdm(val_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                val_losses.append(loss.item())
                val_metrics_list.append(details['acc'].item())

            val_loss, val_metrics = np.mean(val_losses), np.mean(val_metrics_list)
            self.log_writer.add_scalar('val/loss', val_loss, global_step=self.global_step)
            self.log_writer.add_scalar('val/metrics', val_metrics, global_step=self.global_step)

In [ ]:
mnist_train = MNISTDataset(images_dir_path='./mnist_data/train/',
                           description_csv_path='./mnist_data/train.csv')
mnist_val = MNISTDataset(images_dir_path='./mnist_data/val/',
                         description_csv_path='./mnist_data/val.csv')

model = ResNet18()
model_sate_path = 'pretrained_model.pt'
model.load_state_dict(torch.load(model_sate_path, map_location='cpu'))

opt = optim.SGD(model.parameters(), lr=1e-2)

trainer = Trainer(model=model, optimizer=opt, train_dataset=mnist_train,
                  val_dataset=mnist_val, tboard_log_dir='./tboard_logs/exp3')

In [ ]:
trainer.train(5)

In [ ]:
%tensorboard --logdir ./tboard_logs